In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


train=pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')



* [1. EDA](#1.)
* [2. drop + value_counts](#2.)
* [3. train](#3.)
* [4. sub](#4.)




<a id="1."></a>
## 1. EDA

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:

data = train.copy()
data = data[data['Cover_Type'] < train['Cover_Type'].quantile(0.99)]
plt.figure(figsize=(15,10))
plt.title("Cover_Type",fontsize=15)
sns.distplot(data['Cover_Type'])
plt.show()

In [ ]:
plt.figure(figsize=(14,5))
target_values = train['Cover_Type'].value_counts()
sns.barplot(x=target_values.index, y=target_values.values,linewidth=1.5, facecolor='blue',
                 errcolor=".2", edgecolor=".2")
plt.title("Target unique values", fontdict={'fontsize':20})
plt.show()

In [ ]:
test.shape,train.shape

In [ ]:
train['Cover_Type'].unique()

In [ ]:
train[train['Cover_Type']==1].shape,train[train['Cover_Type']==2].shape,train[train['Cover_Type']==3].shape

In [ ]:
train[train['Cover_Type']==6].shape,train[train['Cover_Type']==7].shape,train[train['Cover_Type']==4].shape,train[train['Cover_Type']==5].shape

In [ ]:
train.describe()

In [ ]:
k = 10
f,ax = plt.subplots(figsize=(11, 11))
cols = train.corr().nlargest(k, 'Cover_Type')['Cover_Type'].index
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
train.isnull().sum()


<a id="2."></a>
## 2. drop + value_counts

In [ ]:
y=train.pop('Cover_Type')
id_=train.pop('Id')
id_test=test.pop('Id')

In [ ]:
from tqdm import tqdm
tr_te = pd.concat([train,test])

num_cols = [c for c in train.columns]

for f in tqdm(num_cols):
    tr_te[f+'_counts'] = tr_te[f].map(pd.concat([train[f], test[f]], axis=0).value_counts().to_dict(), na_action='ignore')
    tr_te[f+'_counts'] = tr_te[f+'_counts'].fillna(1)


In [ ]:
train_df = tr_te[0:train.shape[0]]
test_df = tr_te[train.shape[0]:]

In [ ]:
del train,test

In [ ]:
train_df.head(2)


<a id="3."></a>
## 3. train

In [ ]:
cat_params = {
    'iterations': 15000,
    'learning_rate': 0.03,
    'od_wait': 1000,
    'depth': 7,
    'task_type' : 'GPU',
    'devices' : '0',
    'verbose' : 1000
}

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier( **cat_params)
model.fit(train_df, y)





<a id="4."></a>
## 4. submission

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
sub['Cover_Type'] =model.predict(test_df)
sub.to_csv("submission.csv",index=False)
sub.head()